In [1]:
from pathlib import Path

In [2]:
import re

In [3]:
!head -150 IDMS_RECORDS_cpy.txt | tail -50

0          1        5,473        3,664        1,809            0            0                                                        
           2        5,473        3,664        1,809            0            0                                                        
           3        5,473        3,664        1,809            0            0                                                        
           4        5,473        3,664        1,809            0            0                                                        
           5        5,473        3,664        1,809            0            0                                                        
           6        5,473        3,664        1,809            0            0                                                        
           7        5,473        3,664        1,809            0            0                                                        
           8        5,473        3,664        1,809           

# Initial cleaning

In [4]:
# !head -50 IDMS_RECORDS_cpy.txt

# p = Path( "2022-10-14_Whitespace_stripped.txt" )

# raw_text = p.read_text(encoding='latin1')

# len( raw_text )

# print( raw_text[:1000] )

# 

# p = re.compile( r'^1REPORT.*?^0$', flags = re.MULTILINE | re.DOTALL )

# len( p.findall( raw_text ) )

# removed_headers = p.sub( "", raw_text )

# len( removed_headers )

In [5]:
out = Path( "2022-10-14_Stripped_IDMS_RECORDS.txt" )

In [6]:
#out.write_text( removed_headers )

In [7]:
# reload after some hand-editing
removed_headers = out.read_text()

In [8]:
len( removed_headers )

15348439

In [9]:
lines = removed_headers.split( '\n' )

In [10]:
len( lines )

171643

In [11]:
p = re.compile( r'^\s+' )

In [12]:
p.findall( lines[0] )

[' ']

In [13]:
from collections import Counter

In [14]:
[ i for i, _ in enumerate( lines ) if len( p.findall( _ ) ) == 0 ]

[171642]

In [15]:
lines = [ _ for _ in lines if _ != "" ]

In [16]:
len( lines )

171642

In [17]:
[ i for i, _ in enumerate( lines ) if len( p.findall( _ ) ) == 0 ]

[]

In [18]:
leading_space_count = [ len( p.findall( _ )[0] ) for _ in lines if len( p.findall( _ ) ) > 0 ]

In [19]:
Counter( leading_space_count )

Counter({1: 3661,
         5: 18569,
         12: 31763,
         14: 2902,
         9: 31042,
         26: 13934,
         11: 62916,
         13: 63,
         10: 6756,
         27: 3,
         22: 33})

# Create grouping variables based on position

In [20]:
table_index = -1
synonym_index = -1
table_indices = [0]*len( lines )
synonym_indices = [0]*len( lines )
modifier_target_lines = [0]*len( lines )
modified_line = -1
modifier_line_prefix = " "*26

for i in range( len( lines ) ):
    if lines[i][1] != ' ':
        table_index += 1
        synonym_index = -1
    if lines[i].startswith( '     RECORD'):
        synonym_index += 1
    if modified_line < 0 and lines[i].startswith( modifier_line_prefix ):
        # first time we're seeing a modifier?
        # Record the line that is being modified
        modified_line = i - 1
    if modified_line > 0 and not lines[i].startswith( modifier_line_prefix ):
        modified_line = -1

    modifier_target_lines[ i ] = modified_line
    table_indices[ i ] = table_index
    synonym_indices[ i ] = synonym_index

In [21]:
#table_indices

In [22]:
len( leading_space_count ) == len( lines )

True

In [23]:
import pandas as pd

In [24]:
lines_df = pd.DataFrame( dict(
    leading_space_count = leading_space_count,
    table_index = table_indices,
    synonym_index = synonym_indices,
    modifies_line = modifier_target_lines,
    line = lines ))

In [25]:
lines_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171642 entries, 0 to 171641
Data columns (total 5 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   leading_space_count  171642 non-null  int64 
 1   table_index          171642 non-null  int64 
 2   synonym_index        171642 non-null  int64 
 3   modifies_line        171642 non-null  int64 
 4   line                 171642 non-null  object
dtypes: int64(4), object(1)
memory usage: 6.5+ MB


In [26]:
lines_df[ lines_df[ 'modifies_line' ] > 0]

,leading_space_count,table_index,synonym_index,modifies_line,line
58,26,1,0,57,REDEFINES ...
60,26,1,0,59,OCCURS 4
70,26,1,0,69,VALUE ...
72,26,1,0,71,VALUE ...
122,26,2,0,121,REDEFINES ...
...,...,...,...,...,...
171557,26,3659,0,171556,REDEFINES ...
171569,26,3659,0,171568,OCCURS 5
171617,26,3660,0,171616,REDEFINES ...
171624,26,3660,0,171623,REDEFINES ...


In [27]:
lines_df[ 'modifies_line' ].value_counts().value_counts()

1         12728
2           245
5            22
4            14
6            13
9            12
10           10
3            10
7             9
8             8
11            6
13            2
157705        1
18            1
Name: modifies_line, dtype: int64

In [28]:
lines_df[ lines_df['leading_space_count'] == 1 ]

,leading_space_count,table_index,synonym_index,modifies_line,line
0,1,0,-1,-1,ABSDOMN VER 1 ...
11,1,1,-1,-1,ACCRLINK VER 1 ...
104,1,2,-1,-1,ACCRLINK VER 2 ...
168,1,3,-1,-1,ACCT VER 1 ...
184,1,4,-1,-1,ACCTG-CNTL VER 1 ...
...,...,...,...,...,...
171288,1,3656,-1,-1,502-REPT-DTL-RCRD VER 1 ...
171366,1,3657,-1,-1,502-REPT-DTL-RCRD VER 2 ...
171433,1,3658,-1,-1,502-REPT-DTL-RCRD VER 4 ...
171505,1,3659,-1,-1,502-TEMP-DTL-RCRD VER 1 ...


In [29]:
text_clumps = lines_df.groupby( 'leading_space_count' ).apply( lambda group: group['line'].to_list() )

In [30]:
text_clumps.info()

<class 'pandas.core.series.Series'>
Int64Index: 11 entries, 1 to 27
Series name: None
Non-Null Count  Dtype 
--------------  ----- 
11 non-null     object
dtypes: object(1)
memory usage: 176.0+ bytes


In [31]:
[ len( _ ) for _ in text_clumps ]

[3661, 18569, 31042, 6756, 62916, 31763, 63, 2902, 33, 13934, 3]

# TABLE NAMES

In [32]:
text_clumps[1][:20]

[' ABSDOMN                          VER     1    32      D                                    10/15/87 09/29/87',
 ' ACCRLINK                         VER     1   176      D                                    03/09/98 02/21/84',
 ' ACCRLINK                         VER     2   176      D                                    05/15/98 05/15/98',
 ' ACCT                             VER     1    40      S                                    09/24/87 09/21/87',
 ' ACCTG-CNTL                       VER     1   100      D                                    02/07/89 05/26/87',
 ' ACH-RCRD                         VER     1   148      D                                             06/22/84',
 ' ACQD-PROP-MAP-RCRD               VER     1   141      C                                    07/31/87 06/30/87',
 ' ACQD-PROP-QUEUE-RCRD             VER     1   260      C                                    01/20/93 06/30/87',
 ' ACQD-PROP-SALES-RCRD             VER     1   286      D                              

In [33]:
Counter( [ len(_) for _ in text_clumps[1] ] )

Counter({109: 3661})

In [34]:
p = re.compile( r'\s(\S+)\s+VER\s+(\d+)' )

In [35]:
table_info_list = [ p.match( _ ).groups(1) for _ in text_clumps[1] ]

In [36]:
len( table_info_list )

3661

In [37]:
table_info_list[:10]

[('ABSDOMN', '1'),
 ('ACCRLINK', '1'),
 ('ACCRLINK', '2'),
 ('ACCT', '1'),
 ('ACCTG-CNTL', '1'),
 ('ACH-RCRD', '1'),
 ('ACQD-PROP-MAP-RCRD', '1'),
 ('ACQD-PROP-QUEUE-RCRD', '1'),
 ('ACQD-PROP-SALES-RCRD', '1'),
 ('ACQD-PROP-SOLD', '1')]

In [38]:
table_names, table_vers = zip(*table_info_list )

In [39]:
created_dates = pd.to_datetime( [ _[101:109] for _ in text_clumps[1] ] )

In [40]:
len( created_dates )

3661

In [41]:
updated_dates = pd.to_datetime( [ _[92:100] for _ in text_clumps[1] ], errors='coerce' )

In [42]:
updated_dates

DatetimeIndex(['1987-10-15', '1998-03-09', '1998-05-15', '1987-09-24',
               '1989-02-07',        'NaT', '1987-07-31', '1993-01-20',
               '1994-09-19', '1991-02-22',
               ...
               '2000-01-28', '1992-06-03', '1989-11-15', '1994-09-16',
               '1983-09-15', '2002-12-09',        'NaT', '2002-03-14',
               '2002-12-09', '2002-03-14'],
              dtype='datetime64[ns]', length=3661, freq=None)

In [43]:
table_info = pd.DataFrame( data=dict(
    table_names=table_names,
    table_vers=table_vers,
    created_dates=created_dates,
    updated_dates= updated_dates
))

In [44]:
table_info.head(20)

,table_names,table_vers,created_dates,updated_dates
0,ABSDOMN,1,1987-09-29,1987-10-15
1,ACCRLINK,1,1984-02-21,1998-03-09
2,ACCRLINK,2,1998-05-15,1998-05-15
3,ACCT,1,1987-09-21,1987-09-24
4,ACCTG-CNTL,1,1987-05-26,1989-02-07
5,ACH-RCRD,1,1984-06-22,NaT
6,ACQD-PROP-MAP-RCRD,1,1987-06-30,1987-07-31
7,ACQD-PROP-QUEUE-RCRD,1,1987-06-30,1993-01-20
8,ACQD-PROP-SALES-RCRD,1,1994-09-01,1994-09-19
9,ACQD-PROP-SOLD,1,1989-04-26,1991-02-22


# 5's are for metadata

In [45]:
text_clumps[5][:20]

['     PREPARED BY          AWMC11',
 '     REVISED BY           AWMC11',
 '     DESCRIPTION          USER DOMAIN',
 '     RECORD               ABSDOMN                                  VER     1 D  SUFFIX -4541',
 '     PREPARED BY          AA-KGM',
 '     REVISED BY           ASL034',
 '     DESCRIPTION          ACCRUAL LINK RECORD',
 '     RFA-ID               H-3973',
 '     RFA-ID               H-3581A',
 '     DEFINITION',
 '     RECORD               ACCRLINK                                 VER     1 D  PREFIX W019-',
 '     PREPARED BY          ASL70M',
 '     REVISED BY           ASL70M',
 '     DESCRIPTION          ACCRUAL LINK RECORD',
 '     RFA-ID               H-3973',
 '     RFA-ID               H-3581A',
 '     SAME AS              ACCRLINK                                 VER     1',
 '     DEFINITION',
 '     RECORD               ACCRLINK                                 VER     2 D  PREFIX W019-',
 '     PREPARED BY          ASL052']

In [46]:
p = re.compile( r'     (\S+)' )

In [47]:
Counter( [ p.match( _ ).groups(1)[0] for _ in text_clumps[5] ] )

Counter({'PREPARED': 3629,
         'REVISED': 3032,
         'DESCRIPTION': 3176,
         'RECORD': 4126,
         'RFA-ID': 729,
         'DEFINITION': 1166,
         'SAME': 573,
         'IN': 1737,
         'COMMENT': 203,
         'PURPOSE': 63,
         'PRIMARY': 27,
         'FMHA-TRANS': 19,
         'LANGUAGE': 44,
         'SUB-SYSTEM': 2,
         'MODE': 39,
         'EGEN': 1,
         'USER': 1,
         'MEDIUM': 1,
         'FREQUENCY': 1})

## Pull DESCRIPTION one-liners

In [48]:
p = re.compile( r'^ (\S+)\s+VER\s+(\d+).*?DESCRIPTION\s+(.*?)$', flags=re.DOTALL | re.MULTILINE ) 

In [49]:
desc_table_names, desc_table_vers, desc_descs = zip( *p.findall( removed_headers ) )

In [50]:
len( desc_table_names )

3184

In [51]:
len( desc_table_vers )

3184

In [52]:
len( desc_table_names) 

3184

In [53]:
descs = pd.DataFrame( dict(
    table_names = desc_table_names,
    table_vers = desc_table_vers,
    descriptions = desc_descs ))

In [54]:
table_info.shape

(3661, 4)

In [55]:
descs.shape

(3184, 3)

In [56]:
merged_table_info = table_info.merge( descs, on=['table_names', "table_vers"], how='left' )

In [57]:
merged_table_info

,table_names,table_vers,created_dates,updated_dates,descriptions
0,ABSDOMN,1,1987-09-29,1987-10-15,USER DOMAIN
1,ACCRLINK,1,1984-02-21,1998-03-09,ACCRUAL LINK RECORD
2,ACCRLINK,2,1998-05-15,1998-05-15,ACCRUAL LINK RECORD
3,ACCT,1,1987-09-21,1987-09-24,DFIS ACCOUNT
4,ACCTG-CNTL,1,1987-05-26,1989-02-07,ACCOUNTING MONTH ACTIVITY TOTALS
...,...,...,...,...,...
3656,502-REPT-DTL-RCRD,1,1984-06-28,2002-12-09,COPY MODULE FOR 502-REPORT
3657,502-REPT-DTL-RCRD,2,2003-06-13,NaT,COPY MODULE FOR 502-REPORT
3658,502-REPT-DTL-RCRD,4,2002-03-12,2002-03-14,COPY MODULE FOR 502-REPORT
3659,502-TEMP-DTL-RCRD,1,1989-01-30,2002-12-09,WORK RECORD FOR 502 REPORT


## Pull synonym table names, prefixes and suffixes

In [58]:
suffix_df = lines_df[ lines_df['leading_space_count'] == 5 ]

In [59]:
suffix_df.shape

(18569, 5)

In [60]:
suffix_df = suffix_df[ suffix_df['line'].str.startswith( '     RECORD' ) ]

In [61]:
suffix_df = suffix_df.copy()

In [62]:
suffix_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4126 entries, 4 to 171583
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   leading_space_count  4126 non-null   int64 
 1   table_index          4126 non-null   int64 
 2   synonym_index        4126 non-null   int64 
 3   modifies_line        4126 non-null   int64 
 4   line                 4126 non-null   object
dtypes: int64(4), object(1)
memory usage: 193.4+ KB


In [63]:
name_start_pos = 26
name_end_pos = 67
affix_type_start_pos = 80
affix_type_end_pos = 86
affix_start_pos = 87

In [64]:
suffix_df['synonym_name'] = [ _[26:67].strip() for _ in suffix_df['line'].values ]

In [65]:
suffix_df['affix_type'] = [ _[80:86] if len(_) > 80 else "" for _ in suffix_df['line'].values  ]

In [66]:
suffix_df['affix'] = [ _[87:] if len(_) > 80 else "" for _ in suffix_df['line'].values  ]

In [67]:
suffix_df.tail(40)

,leading_space_count,table_index,synonym_index,modifies_line,line,synonym_name,affix_type,affix
169605,5,3636,2,-1,RECORD SYNONYM WZ-PAY-REAP-WPR ...,WZ-PAY-REAP-WPR,PREFIX,WPR-
169629,5,3636,3,-1,RECORD SYNONYM WZ-PAY-REAP-PR ...,WZ-PAY-REAP-PR,PREFIX,PR-
169659,5,3637,0,-1,RECORD WZ-STATE ...,WZ-STATE,,
169674,5,3637,1,-1,RECORD SYNONYM WZ-STATE-STA ...,WZ-STATE-STA,PREFIX,STA-
169689,5,3637,2,-1,RECORD SYNONYM WZ-STATE-TPE ...,WZ-STATE-TPE,PREFIX,TPS-
169707,5,3638,0,-1,RECORD WZ-TRAN-FIN ...,WZ-TRAN-FIN,,
169756,5,3638,1,-1,RECORD SYNONYM WZ-TRAN-FIN-WKP ...,WZ-TRAN-FIN-WKP,PREFIX,WTF-
169805,5,3638,2,-1,RECORD SYNONYM WZ-TRAN-FIN-DSC ...,WZ-TRAN-FIN-DSC,PREFIX,DTF-
169854,5,3638,3,-1,RECORD SYNONYM WZ-TRAN-FIN-PRC ...,WZ-TRAN-FIN-PRC,PREFIX,PTF-
169903,5,3638,4,-1,RECORD SYNONYM WZ-TRAN-FIN-TF ...,WZ-TRAN-FIN-TF,PREFIX,TF-


In [68]:
temp = list( suffix_df['line'] )

In [69]:
temp[0][67:76]

'VER     1'

In [70]:
temp[0].index("VER")

67

In [71]:
#p = re.compile( r' ' )

In [72]:
#suffix_df.groupby( 'table_index' ).apply( , axis=1, result_type='expand' )

# 9's indicate "COPIED IN PROGRAM", "FILE", AND "SUBSCHEMA"

In [73]:
#text_clumps[9][-200:]
text_clumps[9][-20:]

['         SUBSCHEMA            SSPMF03M  OF SCHEMA SCMMF01M             VER     1',
 '         SUBSCHEMA            SSPMF03P  OF SCHEMA SCMMF01P             VER     1',
 '         SUBSCHEMA            XSSPMF03  OF SCHEMA XSCMMF01             VER     1',
 '         COPIED IN PROGRAM    SBMDD010                                 VER     1',
 '         COPIED IN PROGRAM    SCMDD021                                 VER     1',
 '         COPIED IN PROGRAM    SBDLD142                                 VER     1',
 '         COPIED IN PROGRAM    SDDLD355                                 VER     5',
 '         COPIED IN PROGRAM    SDDLD355                                 VER     3',
 '         COPIED IN PROGRAM    SBDLD101                                 VER     6',
 '         COPIED IN PROGRAM    SDDLD355                                 VER     1',
 '         COPIED IN PROGRAM    SBDLD101                                 VER     4',
 '         COPIED IN PROGRAM    SDDLD355                         

In [74]:
p = re.compile( r'\s+(\S+)' )

In [75]:
Counter( [ p.match( _ ).groups(1)[0] for _ in text_clumps[9] if p.match( _ ) ] )

Counter({'COPIED': 29058, 'SUBSCHEMA': 1834, 'FILE': 70, 'LANGUAGE': 80})

In [76]:
sorted( [ _ for _ in text_clumps[9] if "FILE" in _ ] )

['         FILE                 ADPBABAA                                 VER     1',
 '         FILE                 ADPBABAA                                 VER     1',
 '         FILE                 ADPBADAC                                 VER     1',
 '         FILE                 APIW-CARDIN                              VER     1',
 '         FILE                 APIW-FIND                                VER     1',
 '         FILE                 APIW-PISALL                              VER     1',
 '         FILE                 APIW-PITAPE                              VER     1',
 '         FILE                 APIW-PITAPE                              VER     1',
 '         FILE                 APIW-PITAPE                              VER     1',
 '         FILE                 APIW-PIWORK                              VER     1',
 '         FILE                 APIW-PIWORK                              VER     1',
 '         FILE                 APIW-PIWORK                      

In [77]:
[ _ for _ in text_clumps[9] if "LANGUAGE" in _ ]

['         LANGUAGE             ASSEMBLER',
 '         LANGUAGE             FORTRAN',
 '         LANGUAGE             COBOL',
 '         LANGUAGE             PL/I',
 '         LANGUAGE             COBOL',
 '         LANGUAGE             PL/I',
 '         LANGUAGE             ASSEMBLER',
 '         LANGUAGE             ASSEMBLER',
 '         LANGUAGE             ASSEMBLER',
 '         LANGUAGE             FORTRAN',
 '         LANGUAGE             ASSEMBLER',
 '         LANGUAGE             FORTRAN',
 '         LANGUAGE             ASSEMBLER',
 '         LANGUAGE             FORTRAN',
 '         LANGUAGE             ASSEMBLER',
 '         LANGUAGE             FORTRAN',
 '         LANGUAGE             ASSEMBLER',
 '         LANGUAGE             FORTRAN',
 '         LANGUAGE             ASSEMBLER',
 '         LANGUAGE             FORTRAN',
 '         LANGUAGE             ASSEMBLER',
 '         LANGUAGE             FORTRAN',
 '         LANGUAGE             ASSEMBLER',
 '         LANGUAGE   

# 10's through 12's are data elements

In [78]:
text_clumps[10][-10:]

['          14500   05 DTE-LST-CNCLTN-RDA                       9(6)                             DISPLAY',
 '          14600   05 PRIN-INSTLMT-BOND-ACCTS-RDA              S9(7)V99                         COMP-3',
 '          14700   05 INSTLMT-OC-DATA-320-RDA                                                   DISPLAY',
 '          14800    10 DTE-INSTLMT-DUE-320-RDA                 9(6)                             DISPLAY',
 '          14900    10 INSTLMT-DUE-AMT-320-RDA                 S9(06)V99                        COMP-3',
 '          15000    10 INSTLMT-TYP-CDE-320-RDA                 9(1)                             DISPLAY',
 '          15100    10 MTURD-CDE-DIR-RE-320-RDA                9(1)                             DISPLAY',
 '          15200   05 RCVB-AMT-DUE-RDA                         S9(09)V99                        COMP-3',
 '          15300   05 RCVB-AMT-PD-RDA                          S9(09)V99                        COMP-3',
 '          15400   05 FILLER            

In [79]:
text_clumps[11][-10:]

['           4600   05 W502-RGSTR-TYP-LST-CHNG                  9(2)                             DISPLAY',
 '           4700   05 FILLER                                   X(0014)                          DISPLAY',
 '           4800   05 W502-DTE-APPRVL-OBLGN                    9(5)                             DISPLAY',
 '           4900   05 W502-ETHNCTY-TYP-CDE                     X                                DISPLAY',
 '           5000   05 W502-RACE-TYP-CDE                        X                                DISPLAY',
 '           5100   05 W502-RACE-DTRMNTN-CDE                    X                                DISPLAY',
 '           5200   05 W502-ST-CDE                              9(2)                             DISPLAY',
 '           5300   05 FILLER                                   X(0002)                          DISPLAY',
 '           5400   05 W502-CR-REPT-FEE-CDE                     9(1)                             DISPLAY',
 '           5500   05 W502-PROJ-NBR-

In [80]:
text_clumps[12][-10:]

['            900    10 W502-PPB-CDE-1ST                        9                                DISPLAY',
 '            100   05 W502-RCRD-TYP                            9(2)                             DISPLAY',
 '            200   05 W502-RGSTR-TYP-CDE                       9(2)                             DISPLAY',
 '            300   05 W502-MO-OBLGN                            9(2)                             DISPLAY',
 '            400   05 W502-FY                                  9(2)                             DISPLAY',
 '            500   05 W502-APROPTN-MAJ-CLS-CDE                                                  DISPLAY',
 '            600    10 W502-APROPTN-CDE                        9(2)                             DISPLAY',
 '            700    10 W502-MAJ-CLS                            9(2)                             DISPLAY',
 '            800   05 W502-PROG-PLANG-BDGTG-CDE                                                 DISPLAY',
 '            900    10 W502-PPB-CDE-

In [81]:
all_data_elements = [ *text_clumps[10], *text_clumps[11], *text_clumps[12] ]

In [82]:
len( all_data_elements )

101435

## Get a count of all elements in the base (non-synonym version) of a table

In [83]:
all_elements = lines_df[ (lines_df['leading_space_count'] >= 10) & (lines_df['leading_space_count'] <= 12) & (lines_df['synonym_index'] == 0) ] 

In [84]:
all_elements.shape

(89288, 5)

In [85]:
all_elements.groupby( 'table_index' )['line'].count()

table_index
0        6
1       48
2       48
3        7
4       15
        ..
3656    57
3657    57
3658    57
3659    55
3660    55
Name: line, Length: 3601, dtype: int64

# 13's are data elements as well as descriptions

In [86]:
len(text_clumps[13])

63

In [87]:
text_clumps[13][:20]

['             10   05 TRNSCTN-CDE-UPDTE                        X(2)                             DISPLAY',
 '             20   05 CASE-NBR                                 X(15)                            DISPLAY',
 '             30   05 OK-TO-PROC-CDE                           X(1)                             DISPLAY',
 '             40   05 FD-CDE-2                                 X(2)                             DISPLAY',
 '             50   05 LN-TYP-ABBR                              X(2)                             DISPLAY',
 '             60   05 LN-NBR                                   X(2)                             DISPLAY',
 '             70   05 COLLCTN-TYP-CDE-MISC                     X(2)                             DISPLAY',
 '             80   05 FILLER                                   X(0005)                          DISPLAY',
 '             90   05 PYMT-AMT                                 X(7)                             DISPLAY',
 '             1000 INT-RATE-CTGRY IS

In [88]:
data_elements_13 = [ _ for _ in text_clumps[13] if _[16] == ' ' ]

In [89]:
len( data_elements_13 )

9

In [90]:
data_elements_13

['             10   05 TRNSCTN-CDE-UPDTE                        X(2)                             DISPLAY',
 '             20   05 CASE-NBR                                 X(15)                            DISPLAY',
 '             30   05 OK-TO-PROC-CDE                           X(1)                             DISPLAY',
 '             40   05 FD-CDE-2                                 X(2)                             DISPLAY',
 '             50   05 LN-TYP-ABBR                              X(2)                             DISPLAY',
 '             60   05 LN-NBR                                   X(2)                             DISPLAY',
 '             70   05 COLLCTN-TYP-CDE-MISC                     X(2)                             DISPLAY',
 '             80   05 FILLER                                   X(0005)                          DISPLAY',
 '             90   05 PYMT-AMT                                 X(7)                             DISPLAY']

In [91]:
all_data_elements.extend( data_elements_13 )

In [92]:
len( all_data_elements )

101444

## Locations

* [0:15] - declaration step
* [15:62] - (\s+)(\d+) (\s+)
* [62:95] - data format
* [95:] - end

In [93]:
all_elements.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89288 entries, 5 to 171641
Data columns (total 5 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   leading_space_count  89288 non-null  int64 
 1   table_index          89288 non-null  int64 
 2   synonym_index        89288 non-null  int64 
 3   modifies_line        89288 non-null  int64 
 4   line                 89288 non-null  object
dtypes: int64(4), object(1)
memory usage: 4.1+ MB


In [94]:
p = re.compile( r'^\s+(\d+)(\s+)(\d+) (\S+)' )#\s+(\S+)?\s+(\S)$' )
#p = re.compile( r'^\s+(\d+)\s+(\d+) (\S+)\s+(\S+)?\s+(\S)$' )

In [95]:
p.match( all_data_elements[0] ).groups()

('10000', '   ', '05', 'AW-U-YR')

In [97]:
declaration_steps, indent, indent_number, field_name  = zip( *all_elements['line'].transform( lambda l: p.match( l ).groups() ).values )

In [98]:
data_types = all_elements['line'].transform( lambda l: l[62:95].strip() )

In [99]:
# not sure to what to call the "DISPLAY" etc that brings up the rear of these data field lines
end_types = all_elements['line'].transform( lambda l: l[95:].strip() )

In [100]:
len( data_types )

89288

In [101]:
len( indent )

89288

In [102]:
indent = [ len(_) for _ in indent ]

In [103]:
data_fields = pd.DataFrame(
    data = dict(
        declaration_step = declaration_steps,
        indent_space_count = indent,
        indent_number = indent_number,
        field_name = field_name,
        data_type = data_types,
        end = end_types
    ),
    index = all_elements.index
)

In [105]:
data_fields.sample( 10 )

,declaration_step,indent_space_count,indent_number,field_name,data_type,end
165300,500,4,10,ID-NBR-OBLR-8P12,X(10),DISPLAY
83001,3400,4,10,W011-DTE-JULIAN-GRP-MFH,,DISPLAY
124329,1000,4,10,W236-OBLGN-EXPNTR-CDE-MFH,,DISPLAY
128982,2300,3,05,CRC-FO-APPEAL-INDCTR,X(1),DISPLAY
146835,700,4,10,TYP-SRVC-CDE-BI57M,99,DISPLAY
83321,3100,4,10,W055-APROPTN-CDE,9(2),DISPLAY
33621,600,4,10,FD-CDE-4TH,9(1),DISPLAY
71256,14700,3,05,D014-ACQD-PROP-CST,S9(9)V99,COMP-3
14504,12800,3,05,PR-SBSY-AMT,S9(5),COMP-3
161125,1600,3,05,LN-NBR-4VP,9(2),DISPLAY


In [108]:
data_fields[ ['indent_space_count', 'indent_number' ] ].value_counts()

indent_space_count  indent_number
3                   05               53625
4                   10               24046
5                   15                6156
7                   88                2718
3                   03                1375
6                   20                 906
3                   04                 151
                    02                 146
7                   25                 112
4                   07                  49
                    09                   4
dtype: int64

In [110]:
data_fields[ data_fields[ 'indent_number' ] =='09']

,declaration_step,indent_space_count,indent_number,field_name,data_type,end
111534,6100,4,09,DCSTR4,X(4),DISPLAY
111535,6200,4,09,DCSTR5,X(4),DISPLAY
112045,7100,4,09,DCSTR4,X(4),DISPLAY
112046,7200,4,09,DCSTR5,X(4),DISPLAY


In [120]:
unique_data_types = set( data_types )

In [ ]:
all_elements

In [121]:
unique_data_types = sorted( list( unique_data_types ), key=lambda t: ( len(t), t ) )

In [78]:
Path( '2022-10-17_IDMS_COBOL_datatypes.txt' ).write_text( '\n'.join( unique_data_types ) )

5115

# 14's are DEFINITION, COMMENT, and PURPOSE

In [79]:
text_clumps[14][:20]

['              100 THIS RECORD IS USED TO PASS TO AND',
 '              200 FROM INFORMATION REQUIRED TO COMPLETE',
 '              300 AN ACCRUAL',
 '              100 THIS RECORD IS USED TO PASS TO AND',
 '              200 FROM INFORMATION REQUIRED TO COMPLETE',
 '              300 AN ACCRUAL',
 '              100 THE AMOUNT IN THAT DFIS ACCOUNT.',
 '              100 AT THE END OF EACH ACCOUNTING MONTH BEFORE THE',
 '              200 STATE-CNTL AND THE CYCLE-CNTL RECORDS ARE RESET',
 '              300 TO ZERO FOR THE NEXT MONTH, THE ACTIVITY CONTROL',
 '              400 TOTALS FOR THE COMPLETED MONTH WILL BE MOVED',
 '              500 INTO THIS 15 MONTH ACCTG-CNTL FILE.',
 '              100 ACQUIRED PROPERTY QUEUE RECORD',
 '              100 STANDARD RECORD DESCRIPTION FOR ACQUIRED',
 '              200 PROPERTY TAPE FILE',
 '              100 PICTURE AND USAGE FOR SOME ELEMENTS HAVE BEEN OVERRIDDEN',
 '              200 TO CREATE RECORD AS ESTABLISHED DURING CONVERSION',
 '

In [80]:
comment_lines = lines_df[ lines_df['leading_space_count'] == 14 ]

In [81]:
comments = comment_lines.groupby( 'table_index' ).apply( lambda group: " ".join( _[18:] for _ in group['lines'].values  ) )

In [82]:
comments[2]

'THIS RECORD IS USED TO PASS TO AND FROM INFORMATION REQUIRED TO COMPLETE AN ACCRUAL'

In [83]:
comments

table_index
1       THIS RECORD IS USED TO PASS TO AND FROM INFORM...
2       THIS RECORD IS USED TO PASS TO AND FROM INFORM...
3                        THE AMOUNT IN THAT DFIS ACCOUNT.
4       AT THE END OF EACH ACCOUNTING MONTH BEFORE THE...
7                          ACQUIRED PROPERTY QUEUE RECORD
                              ...                        
3656    CHANGED BLK-NBR TO  PROJ-CHK-DIGIT CHANGED DTE...
3657    CHANGED BLK-NBR TO  PROJ-CHK-DIGIT CHANGED DTE...
3658    CHANGED BLK-NBR TO  PROJ-CHK-DIGIT CHANGED DTE...
3659    THIS RECORD IS USED FOR A WORKING STORAGE AREA...
3660    THIS RECORD IS USED FOR A WORKING STORAGE AREA...
Length: 1307, dtype: object

In [84]:
comments.name = 'comments'

In [85]:
merged_table_info = merged_table_info.merge( comments, how='left', left_index=True, right_index=True )

In [86]:
merged_table_info = merged_table_info.sort_values( ['created_dates', 'table_names', 'table_vers'] )

In [87]:
merged_table_info.to_csv( '2022-10-17_IDMS_table_descriptions.csv' )

In [88]:
merged_table_info

,table_names,table_vers,created_dates,updated_dates,descriptions,comments
522,DB-STATISTICS,1,1981-08-25,2010-07-16,NaN,NaN
525,DB_STATISTICS,1,1981-08-25,2010-07-16,WORK RECORD FOR DBMG06 MAP,NaN
532,DC-AID-CONDITION-NAMES,1,1981-08-25,2010-07-16,NaN,NaN
1118,MRB-,1,1981-08-25,2010-07-16,NaN,"THIS RECORD IS REQUIRED FOR MAPPING FUNCTIONS,..."
2020,SSCTRL,1,1981-08-25,1994-10-22,NaN,NaN
...,...,...,...,...,...,...
1724,SDDLD560-MAPWORK,4,2020-09-15,2020-09-16,SDDLD560 MAPWORK,NaN
1721,SDDLD560-MAPDAVE,1,2020-09-24,2020-09-24,SDDLD560 MAPWORK,NaN
69,ADPTS601-REC,1,2021-02-19,NaT,UNCLOSED LOANS,NaN
60,ADPTSU01-REC,1,2021-02-19,NaT,GRANT,NaN


# 22's 

In [89]:
text_clumps[22][:20]

['                      EXT PICT  ZZ/ZZ/ZZ',
 '                      EXT PICT  ZZ/ZZ/ZZ',
 '                      EXT PICT  ZZ',
 '                      EXT PICT  ZZ9',
 '                      EXT PICT  ZZ/ZZ/ZZ',
 '                      EXT PICT  ZZ/ZZ/ZZ',
 '                      EXT PICT  ZZ',
 '                      EXT PICT  ZZ9',
 '                      EXT PICT  ZZZ9',
 '                      EXT PICT  ZZZ9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  ZZZ9',
 '                      EXT PICT  ZZZ9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9',
 '                      EXT PICT  Z9']

# 26's: REDEFINES, OCCURS, VALUE

In [78]:
len( text_clumps[26] )

13934

In [90]:
text_clumps[26][:20]

['                          REDEFINES               W019-RQST-AREA-MFH',
 '                          OCCURS            4',
 "                          VALUE                   'A'",
 "                          VALUE                   'U'",
 '                          REDEFINES               W019-RQST-AREA-MFH',
 '                          OCCURS            4',
 "                          VALUE                   'A'",
 "                          VALUE                   'U'",
 '                          VALUE                   0',
 '                          VALUE                   0',
 '                          REDEFINES               W138-CTY-OFC-MAILG',
 "                          VALUE                   '0'",
 "                          VALUE                   '1'",
 '                          OCCURS           65',
 '                          OCCURS           65',
 '                          OCCURS           12',
 '                          OCCURS           12',
 '                   

In [91]:
text_clumps[27][:20]

['                           100   COMMENT DATA',
 '                           100   OPERATOR',
 '                           200   NUMBER']